# Neural network Tuning

In [6]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from keras.wrappers.scikit_learn import KerasClassifier

# part 1 - Tuning batch and epoch

In [7]:
# load the dataset for India diabetes
data = pd.read_csv('diabetes.csv')

In [8]:
#viewing data
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
# split into predictors and response
X = data.iloc[:,0:8] #predictors
Y = data.iloc[:,8] #response

In [10]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))

    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [11]:
# create model
model = KerasClassifier(build_fn=create_my_model)

In [12]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50]

In [13]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)


mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(X, Y)

Epoch 1/10
20/20 [==============================] - 1s 2ms/step - loss: 6.5618 - accuracy: 0.6263
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 3.5815 - accuracy: 0.5117
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 3.0537 - accuracy: 0.5365
Epoch 4/10
20/20 [==============================] - 0s 3ms/step - loss: 2.8116 - accuracy: 0.5234
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 2.5561 - accuracy: 0.5169
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 2.3175 - accuracy: 0.5378
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 2.0686 - accuracy: 0.5469
Epoch 8/10
20/20 [==============================] - 0s 2ms/step - loss: 1.8879 - accuracy: 0.5638
Epoch 9/10
20/20 [==============================] - 0s 2ms/step - loss: 1.7143 - accuracy: 0.5794
Epoch 10/10
20/20 [==============================] - 0s 2ms/step - loss: 1.5758 - accuracy: 0.6302


In [14]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.667969 using {'batch_size': 40, 'epochs': 10}


# part 2- Optimizing Activation Function

In [15]:
# Function to create model
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [16]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [17]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [18]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/50
77/77 [==============================] - 1s 2ms/step - loss: 1.8389 - accuracy: 0.6120
Epoch 2/50
77/77 [==============================] - 0s 1ms/step - loss: 0.6686 - accuracy: 0.6667
Epoch 3/50
77/77 [==============================] - 0s 1ms/step - loss: 0.6568 - accuracy: 0.6536
Epoch 4/50
77/77 [==============================] - 0s 1ms/step - loss: 0.6550 - accuracy: 0.6497
Epoch 5/50
77/77 [==============================] - 0s 2ms/step - loss: 0.6486 - accuracy: 0.6602
Epoch 6/50
77/77 [==============================] - 0s 2ms/step - loss: 0.6357 - accuracy: 0.6758
Epoch 7/50
77/77 [==============================] - 0s 2ms/step - loss: 0.6396 - accuracy: 0.6549
Epoch 8/50
77/77 [==============================] - 0s 2ms/step - loss: 0.6312 - accuracy: 0.6628
Epoch 9/50
77/77 [==============================] - 0s 2ms/step - loss: 0.6343 - accuracy: 0.6641
Epoch 10/50
77/77 [==============================] - 0s 1ms/step - loss: 0.6356 - accuracy: 0.6562
Epoch 11/50
77/77 [

In [19]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.671875 using {'optimizer': 'SGD'}


# How can you improve accuracy of the model?

Frist way

In [20]:
from keras.layers import Dropout
from keras.optimizers import Adam

In [21]:
def create_my_model(optimizer='adam', learning_rate=0.01, dropout_rate=0.0):

    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dropout(dropout_rate))
    mymodel.add(Dense(1, activation='sigmoid'))


    optimizer = Adam(learning_rate=learning_rate)
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel


In [22]:
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [23]:
optimizer = ['SGD', 'Adadelta', 'RMSprop', 'Adagrad', 'Adam']
learning_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.2, 0.4]

In [24]:
parameter_grid = dict(optimizer=optimizer, learning_rate=learning_rate, dropout_rate=dropout_rate)

In [25]:
from keras.callbacks import EarlyStopping

In [26]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [27]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3, verbose=1, scoring='accuracy',
                    return_train_score=True, refit=False)


In [28]:
grid_result = grid.fit(X, Y, callbacks=[early_stopping])

Fitting 3 folds for each of 45 candidates, totalling 135 fits


In [29]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.739583 using {'dropout_rate': 0.0, 'learning_rate': 0.01, 'optimizer': 'Adam'}


Second way

In [30]:
def create_my_model(optimizer='adam'):

    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))

    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [31]:
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10, verbose=0)


In [32]:
optimizer = ['SGD', 'Adadelta', 'RMSprop', 'Adagrad', 'Adam']
param_grid = dict(optimizer=optimizer)

In [33]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)


In [34]:
grid_result = grid.fit(X, Y)

In [35]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.687500 using {'optimizer': 'Adam'}


In [36]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.661458 (0.034987) with: {'optimizer': 'SGD'}
0.458333 (0.144701) with: {'optimizer': 'Adadelta'}
0.656250 (0.038670) with: {'optimizer': 'RMSprop'}
0.460938 (0.019401) with: {'optimizer': 'Adagrad'}
0.687500 (0.008438) with: {'optimizer': 'Adam'}
